# Deep Learning Tutorial 2: Training

Welcome to the second deep learning tutorial!

In this notebook, we are going to apply recurrent neural networks to time series data in order to predict if a hard drive will fail the next day given a series of daily S.M.A.R.T. status observations.

Please note: 

- You can interrupt the training process at any time by clicking on *Kernel* and then *Interrupt*.

- Before you get started, please click on *File* and *Trust Notebook*.

Please execute the following chunks consecutively while reading through it using **Shift + Return**.

## Preparation

We will be using the [Keras](http://keras.io) framework that abstracts away a lot of the details of deep learning. We will only use the [funcational API](https://keras.io/getting-started/functional-api-guide/) due to its expressive power.

In [ ]:
import numpy as np
np.random.seed(42)

from keras.models import Model
from keras.layers import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils.visualize_util import plot, model_to_dot
from IPython.display import SVG

from callbacks import AUCHistory

### Loading our data

Data has been saved in a file named *data.npz*. It can be loaded using *np.load* and then behaves similar to a Python dictionary, where the keys *records* and *labels* corresponds to a numpy array for our input data and our labels, respectively.

In [ ]:
saved = np.load("data.npz")
data = saved["records"]
labels = saved["labels"]

The input data is a 3-dimensional array, where the first dimension corresponds to the number of instances, the second dimension to the number of time steps, and the third dimension to the number of features.

In [ ]:
data.shape

More specifically, 74574 is the number of drives in our data, 90 the number of observations per drive, and 15 features per record. Let us define some variables to make this more obvious.

In [ ]:
n_drives = data.shape[0]
n_records = data.shape[1]
n_features = data.shape[2]

First, we define the input layer, which just takes in our data. It does not contain any logic other than defining the shape of our input. Since we use the functional API, this also means that all matrix shapes in the following layers will be inferred automatically.

In [ ]:
input = Input(shape=(n_records, n_features), name="inputs")

Note that the first dimension, *n_drives*, is automatically inferred.

Now, the fun parts starts. We will add an LSTM layer that builds a representation of each drive by performing the same computation on vectors of size n_features for each n_records.

Remember the unfolding in time computation graph for an RNN

![image](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg)

where $x_t$ are observations for a particular drive, e.g. $x_1$ is the first observation and $x_2$ is the second observation. In our case, we are only interested in the last output $o_t$ where $t = \text{n_records}$. The output of this LSTM will be a vector of size 10. In other words, the LSTM has 10 neurons in the output layer.

In [ ]:
x = LSTM(10)(input)

We're almost done! Let's wire up the 10 output neurons of the LSTM to just a single output neuron using a [Dense](https://keras.io/layers/core/#dense) layer. A Dense layer is just your regular fully connected NN layer.

The output of the dense layer will be $\sigma(x)$, with $\sigma(x) = \frac{1}{1 + exp(-z)}$ where $z$ is just a linear combination of the LSTM outputs, i.e. $\sum\limits w_j x_j$ of the previous layer, where $w_j$ are the learnt weights  for the connection from the LSTM to the Dense layer and $x_j$ is the output of the LSTM. Conveniently, the output of $\sigma(x)$ lies between $0$ and $1$ and matches our target well.

In [ ]:
output = Dense(1, activation='sigmoid', name='output')(x)

Let's wrap up the input and output of our Model.

In [ ]:
model = Model(input=input, output=output)

Now, we will compile our model. Here, we specify two parameters:

- optimizer: an optimizer does all the work for us. Given the input and the computed errors, it decides which direction to take. There are quite a few [optimizers available in Keras](https://keras.io/optimizers/).
- loss: the loss or objective function tells the model how well we are doing on our data. In our case, this is simply binary crossentropy, but in other cases this may be e.g. mean squared error. Note that this function needs to be differentiable because during training we need to be able to compute the weight updates. Hence, we cannot optimize for e.g. ROCAUC directly.

In [ ]:
model.compile(optimizer=Nadam(), loss="binary_crossentropy")

Let's print out a nice plot of our model.

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

Note that *None* simply means that the model does not really care how many instances we input.

## Let's train!

We train using a mini-batch size of 100 instances at a time. This speeds up things, because a mini-batch can be computed in parallel on a GPU. We train for eight epochs, i.e. we go over our training set three times.

In [ ]:
bs = 50

Conveniently, Keras will create a hold-out validation set automatically for us when giving the *validation_split* parameter. Let's set it to 20% of our data. **Please leave *verbose* at 2** in the following call, otherwise your notebook may freeze.

In [ ]:
model.fit(data, labels, verbose=2, nb_epoch=4, batch_size=bs, validation_split=0.2, callbacks=[AUCHistory()])

That wasn't so bad. You can see that our network converges after the second epoch and neither our training loss nor the AUC improve anymore. 

## Task 1: Increase the number of neurons

Maybe the model is simply to small to accomodate patterns in our data? Let's try to increase our neurons!

**Your task** is to:

- Mark this chunk and select *Cell* and then *Run All Above*
- Increase the number of neurons to 20

In [ ]:
input = Input(shape=(n_records, n_features), name="inputs")
x = input

### Your code here
x = LSTM(10)(x)
###

output = Dense(1, activation='sigmoid', name='output')(x)
model1 = Model(input=input, output=output)
model1.compile(optimizer=Nadam(), loss="binary_crossentropy")

In [ ]:
model1.fit(data, labels, verbose=2, nb_epoch=4, batch_size=bs, validation_split=0.2, callbacks=[AUCHistory()])

This is much better! If you want, you can try a different number of neurons.

## Task 2: Add another layer

Let's make our model deeper! This is deep learning after all. Note that our network is already deep in time, i.e. we take into consideration 90 time steps. But we can also make it deeper vertically.

Your task is to

- stack another LSTM layer with 20 neurons on top of the layer we already have.

At each time step, the first LSTM will feed into the second LSTM. This is called stacking.

Note that for this, you have to set *return_sequences=True* in the first LSTM. Do you understand why this is required?

In [ ]:
input = Input(shape=(n_records, n_features), name="inputs")
x = input

### Your code goes here:
x = LSTM(20)(x)
##

output = Dense(1, activation='sigmoid', name='output')(x)
model2 = Model(input=input, output=output)
model2.compile(optimizer=Nadam(), loss="binary_crossentropy")

In [ ]:
SVG(model_to_dot(model2, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
model2.fit(data, labels, verbose=2, nb_epoch=4, batch_size=bs, validation_split=0.2, callbacks=[AUCHistory()])

## Task 3: Change the architecture

This task will be a bit more challenging. We are going to use an additional attribute: the disk's model. However, because this attribute is constant in a disks's time series, we will not add it to LSTM that summarizes the time series.

Instead, we will *merge* the $n$-dimensional vector output of the LSTM with a $m$-dimensional vector, where merging means concatenating the two vectors into a vector of dimensionality $n+m$.

Conveniently, we have already encoded the disk's model as a one-hot vector, i.e. the columns in the following matrix correspond to unique disk models and the rows to individual disks.

In [ ]:
models = saved["models"]

In [ ]:
models.shape

In this task, you have to do the following:

- Use the solution from the previous task.
- Create a second Input with shape `(number_of_models, )`.
- Introduce a [Merge](https://keras.io/getting-started/sequential-model-guide/#the-merge-layer) layer that merges `[x, your_new_input]`. This can be done using the `merge` **function** (not the `Merge` class).
- Modify the `Model` instantiation to take two inputs simultaneously, similar to what you have done in the previous step.

Note that you have to implement all of the above tasks before your model actually works.

In [ ]:
input = Input(shape=(n_records, n_features), name="inputs")

### Your code here

###

output = Dense(1, activation='sigmoid', name='output')(x)
model3 = Model(input=[input, input2], output=output)
model3.compile(optimizer=Nadam(), loss="binary_crossentropy")

In [ ]:
SVG(model_to_dot(model3, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
model3.fit([data, models], labels, verbose=2, nb_epoch=4, batch_size=bs, validation_split=0.2, callbacks=[AUCHistory(2)])

It's still getting better! This concludes the second deep learning tutorial.

If you still have some free time, you are welcome to experiment further with our architecture. Things you may want to try:

- Use GRU instead of LSTM units
- Change the number of neurons in the LSTM
- Introduce regularization such as dropout or l2 norm